### 1. Setting up the Environment

In [1]:
# Install Java
#!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download Spark
#if os.path.isfile("./spark-3.5.1-bin-hadoop3.tgz") == False:
#    !wget -q https://downloads.apache.org/spark/spark-3.5.1/spark-3.5.1-bin-hadoop3.tgz

# Unzip the file
#!tar xf spark-3.5.1-bin-hadoop3.tgz

# Setup environment for Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = '/home/ubuntu/spark-3.5.1-bin-hadoop3'

# Import findspark and load it
#!pip install -q findspark
import findspark
findspark.init()

# Install spark-nlp
#!pip install spark-nlp
import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.common import *
from pyspark.sql.functions import *
from pyspark.sql.functions import lit
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler, VectorAssembler, Imputer, OneHotEncoder, StringIndexer
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator, CrossValidatorModel, TrainValidationSplit, TrainValidationSplitModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.linalg import Vectors, VectorUDT
import pyspark.pandas as ps

# Create Spark session
#from pyspark.sql import SparkSession
# spark = SparkSession.builder\
#        .master("local")\
#        .appName("Spark NLP")\
#        .config('spark.executor.memory', "24g")\
#        .getOrCreate()

# Start Spark Session with Spark NLP
from pyspark.sql import SparkSession
spark = SparkSession.builder \
                    .master("local[*]") \
                    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.17.0") \
                    .appName("MyApp") \
                    .getOrCreate()
# spark = sparknlp.start(gpu=False)

from sklearn.neighbors import NearestNeighbors
import warnings
warnings.filterwarnings("ignore")

/home/ubuntu/anaconda3/lib/python3.11/site-packages/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


:: loading settings :: url = jar:file:/home/ubuntu/spark-3.5.1-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
com.databricks#spark-xml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1d4518f9-895b-41cc-95d1-a76c138f733e;1.0
	confs: [default]
	found com.databricks#spark-xml_2.12;0.17.0 in central
	found commons-io#commons-io;2.11.0 in central
	found org.glassfish.jaxb#txw2;3.0.2 in central
	found org.apache.ws.xmlschema#xmlschema-core;2.3.0 in central
	found org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 in central
:: resolution report :: resolve 261ms :: artifacts dl 21ms
	:: modules in use:
	com.databricks#spark-xml_2.12;0.17.0 from central in [default]
	commons-io#commons-io;2.11.0 from central in [default]
	org.apache.ws.xmlschema#xmlschema-core;2.3.0 from central in [default]
	org.glassfish.jaxb#txw2;3.0.2 from central in [default]
	org.scala-lang.modules#scala-collection-compat_2.12;2.9.0 from central in [default]
	--------

In [2]:
spark

### 2. Import libraries

In [3]:
import time
import json
import pickle
import urllib.request
# import altair as alt
import pandas as pd
from sklearn.metrics import classification_report, fbeta_score
import collections
from collections import Counter
from sklearn.datasets import make_classification
from matplotlib import pyplot
from numpy import where
import imblearn

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import Image, display, HTML
import pydot
from pydotplus import graph_from_dot_data
from sklearn.tree import export_graphviz
from wordcloud import WordCloud

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import ElementTree
import datetime

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, fbeta_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from transformers import BertTokenizer, BertModel
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim

### 3. Load data

In [4]:
train_merged_data_conversations_path = 's3a://capstone210/data/train_merged_data_conversations/'
test_merged_data_conversations_path = 'file:/home/ubuntu/workspace/capstone-210-spring2024/data/test'

train_merged_data_conversations_df = spark.read.parquet(train_merged_data_conversations_path)
test_merged_data_conversations_df = spark.read.parquet(test_merged_data_conversations_path)

24/03/16 18:01:30 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [5]:
train_df_final = train_merged_data_conversations_df
test_df_final = test_merged_data_conversations_df

### 4. Split the data and implement different sampling method

#### 4.1 Undersampling

In [6]:
# function to undersample dataset automatically
def undersample(df, outcome_col, seed=1234):
  # Split dataset based on outcome
  split0_df = df.filter(col(outcome_col) == 0)
  split1_df = df.filter(col(outcome_col) == 1)
  # determine which split is major vs minor
  if (split0_df.count() > split1_df.count()):
    major_df = split0_df
    minor_df = split1_df
  else:
    minor_df = split0_df
    major_df = split1_df
  ratio = major_df.count()/minor_df.count()
  print("Ratio of major vs minor before sampling: {}".format(ratio))
  # Start under-sampling with Spark
  sampled_majority_df = major_df.sample(False, 1/ratio, seed)
  combined_df = sampled_majority_df.unionAll(minor_df)
  print(f"Final sample size: {combined_df.count()}")
  return combined_df

In [7]:
# Perform undersampling technique
df_train_UnderSampled = undersample(train_df_final, outcome_col='label')

Ratio of major vs minor before sampling: 33.05456656346749


Final sample size: 5239


In [8]:
# Define the split ratios (80% train, 20% validation)
train_ratio = 0.7
validation_ratio = 0.3

# Split the data into training and validation sets
df_train_split, df_val_split = df_train_UnderSampled.randomSplit([train_ratio, validation_ratio], seed=42)

# naming convention
df_test_split = test_df_final

In [9]:
# Convert PySpark DataFrame to Pandas DataFrame
df_train_pd = df_train_split.select('merged_text', 'type_conversation', 'label').toPandas()
df_val_pd = df_val_split.select('merged_text', 'type_conversation', 'label').toPandas()
df_test_pd = df_test_split.select('merged_text', 'type_conversation', 'label').toPandas()

#### 4.2 Nearest neighbor sampling

In [10]:
# def nearest_neighbor_sampling(data, k=2):
#     """
#     Perform Nearest Neighbor Sampling (NNS) for a dataset.

#     Parameters:
#     - data: DataFrame or array-like, shape (n_samples, n_features)
#             The input dataset.
#     - k: int, optional (default=2)
#          Number of nearest neighbors to consider for sampling.

#     Returns:
#     - sampled_data: DataFrame or array-like, shape (n_samples, n_features)
#                     The sampled dataset.
#     """

#     # Initialize Nearest Neighbors model
#     nn_model = NearestNeighbors(n_neighbors=k)
#     nn_model.fit(data)

#     # Find nearest neighbors
#     indices = nn_model.kneighbors(data, return_distance=False)[:, 1]

#     # Sample data based on nearest neighbors
#     sampled_data = data.iloc[indices]

#     return sampled_data

In [11]:
# # Extract the features you want to sample from the DataFrame
# features_to_sample = train_df_final[['n_people_in_conversation', 'label']]

# # Convert the DataFrame to a 2D array
# # features_array = features_to_sample.values

# # Apply Nearest Neighbor Sampling
# df_train_NNS = nearest_neighbor_sampling(features_to_sample, k=2)

In [12]:
# # Split data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(input_ids, df['label'], test_size=0.2, random_state=42)

In [13]:
# # Convert PySpark DataFrame to Pandas DataFrame
# df_train_pd = df_train_split.select('merged_text', 'label').toPandas()
# df_val_pd = df_val_split.select('merged_text', 'label').toPandas()
# df_test_pd = df_test_split.select('merged_text', 'label').toPandas()

### 5. Build the model pipeline

In [32]:
import tensorflow as tf

In [33]:
# Tokenize type_conversation
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize input data
type_conv_tokens = tokenizer.batch_encode_plus(df_train_pd['type_conversation'].tolist(), padding=True, truncation=True, max_length=16, return_tensors='tf')
convo_tokens = tokenizer.batch_encode_plus(df_train_pd['merged_text'].tolist(), padding=True, truncation=True, max_length=128, return_tensors='tf')

# # Tokenize text data
# text_tokens = tokenizer(df['merged_text'].tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
# time_tokens = tokenizer(df['conversation_start_time'].astype(str).tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')
# type_tokens = tokenizer(df['type_conversation'].astype(str).tolist(), padding='max_length', truncation=True, max_length=max_length, return_tensors='tf')

# Merge tokenized inputs
# input_ids = tf.concat([text_tokens['input_ids'], time_tokens['input_ids'], type_tokens['input_ids']], axis=1)
# attention_masks = tf.concat([text_tokens['attention_mask'], time_tokens['attention_mask'], type_tokens['attention_mask']], axis=1)
# input_ids = tf.concat([convo_tokens['input_ids'], type_conv_tokens['input_ids']], axis=1)
# attention_masks = tf.concat([convo_tokens['attention_mask'], type_conv_tokens['attention_mask']], axis=1)

In [15]:
# # Tokenize merged_text, conversation_start_time, and type_conversation
# def tokenize_data(df):
#     inputs = tokenizer(df['merged_text'].tolist(), 
#                        df['type_conversation'].tolist(),
#                        padding=True,
#                        truncation='longest_first',
#                        max_length=512,
#                        return_tensors='pt')
#     return inputs

In [16]:
# warnings.filterwarnings("ignore", message="Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.")

In [17]:
# train_inputs = tokenize_data(df_train_pd)

In [18]:
# val_inputs = tokenize_data(df_val_pd)

In [19]:
# test_inputs = tokenize_data(df_test_pd)

In [35]:
class BERTCNNEnsemble(nn.Module):
    def __init__(self, bert_model, cnn_model, num_labels):
        super(BERTCNNEnsemble, self).__init__()
        self.bert_model = bert_model
        self.cnn_model = cnn_model
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(768 + cnn_model.output_size, num_labels)

    def forward(self, input_ids, attention_mask, token_type_ids, type_conv_ids):
        # BERT Model
        outputs = self.bert_model(input_ids, attention_mask, token_type_ids)
        pooled_output = outputs[1]  # Use pooled output (CLS token)
        
        # CNN Model
        cnn_output = self.cnn_model(type_conv_ids)

        # Concatenate and classify
        concat_output = torch.cat((pooled_output, cnn_output), dim=1)
        concat_output = self.dropout(concat_output)
        logits = self.fc(concat_output)
        return logits

class CNNModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, kernel_sizes, output_size):
        super(CNNModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.convs = nn.ModuleList([nn.Conv1d(embedding_dim, output_size, kernel_size) for kernel_size in kernel_sizes])
        self.output_size = output_size * len(kernel_sizes)

    def forward(self, x):
        x = self.embedding(x)  # (batch_size, seq_len, embedding_dim)
        x = x.transpose(1, 2)  # (batch_size, embedding_dim, seq_len)
        x = [conv(x) for conv in self.convs]
        x = [torch.max_pool1d(conv, conv.size(-1)).squeeze(-1) for conv in x]
        x = torch.cat(x, dim=1)
        return x

In [26]:
print(type_conv_tokens['input_ids'][0])

tf.Tensor([  101 18847 24277   102], shape=(4,), dtype=int32)


In [36]:
# Create input tensors
# input_ids = train_inputs['input_ids']
# attention_mask = train_inputs['attention_mask']
# token_type_ids = train_inputs['token_type_ids']
# type_conv_ids = type_conv_tokens['input_ids']
# input_ids = convo_tokens['input_ids']
# attention_mask = convo_tokens['attention_mask']
# token_type_ids = type_conv_tokens['token_type_ids']
# type_conv_ids = type_conv_tokens['input_ids']

input_ids = convo_tokens['input_ids'].numpy()
attention_mask = convo_tokens['attention_mask'].numpy()
type_convo_ids = type_conv_tokens['input_ids'].numpy()
type_convo_att = type_conv_tokens['attention_mask'].numpy()

# Initialize BERT and CNN models
cnn_model = CNNModel(vocab_size=tokenizer.vocab_size, embedding_dim=128, kernel_sizes=[2, 3, 4], output_size=64)

# Initialize the ensemble model
ensemble_model = BERTCNNEnsemble(bert_model, cnn_model, num_labels=2)

# Forward pass
logits = ensemble_model(input_ids, attention_mask, type_convo_ids, type_convo_att)

TypeError: 'int' object is not callable

In [ ]:
ensemble_model.eval()
val_preds = []
val_targets = []

with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, token_type_ids, type_conv_ids, labels = batch
        logits = ensemble_model(input_ids, attention_mask, token_type_ids, type_conv_ids)
        _, predicted = torch.max(logits, 1)
        val_preds.extend(predicted.tolist())
        val_targets.extend(labels.tolist())

# Calculate performance scores
precision = precision_score(val_targets, val_preds)
recall = recall_score(val_targets, val_preds)
f1 = f1_score(val_targets, val_preds)
f_beta = fbeta_score(val_targets, val_preds, beta=0.5)

print("Validation Performance Scores:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("F1 Beta Score:", f_beta)

In [14]:
# Define BERT-CNN ensemble model
class BERTCNNEnsemble(nn.Module):
    def __init__(self, bert_model, cnn_model):
        super(BERTCNNEnsemble, self).__init__()
        self.bert_model = bert_model
        self.cnn_model = cnn_model
        self.fc = nn.Linear(384, 2)  # Adjust input size based on the output of the CNN model

    def forward(self, input_ids, attention_mask, token_type_ids):
        # BERT Model
        outputs = self.bert_model(input_ids, attention_mask, token_type_ids)
        pooled_output = outputs[1]  # Use pooled output (CLS token)

        # CNN Model
        cnn_output = self.cnn_model(pooled_output.unsqueeze(0))

        # Concatenate and classify
        logits = self.fc(cnn_output)
        return logits

In [12]:
# !pip install keras

In [13]:
# !pip install tensorflow

In [15]:
from keras.layers import Input, Conv1D, GlobalMaxPooling1D, Concatenate, Dense

In [16]:
# Define CNN model architecture using PyTorch
class CNNModel(nn.Module):
    def __init__(self, input_size):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(input_size, 128, kernel_size=3)
        self.conv2 = nn.Conv1d(input_size, 128, kernel_size=4)
        self.conv3 = nn.Conv1d(input_size, 128, kernel_size=5)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool1d(kernel_size=3)

    def forward(self, x):
        x1 = self.pool(self.relu(self.conv1(x)))
        x2 = self.pool(self.relu(self.conv2(x)))
        x3 = self.pool(self.relu(self.conv3(x)))
        return torch.cat((x1, x2, x3), dim=2)

In [17]:
# Define training parameters
batch_size = 32
learning_rate = 1e-5
num_epochs = 5

In [18]:
from torch.utils.data import Dataset

In [19]:
# Define dataset class
class CustomDataset(Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __len__(self):
        return len(self.inputs['input_ids'])

    def __getitem__(self, idx):
        input_ids = self.inputs['input_ids'][idx]
        attention_mask = self.inputs['attention_mask'][idx]
        token_type_ids = self.inputs['token_type_ids'][idx]
        label = torch.tensor(self.labels[idx])
        return input_ids, attention_mask, token_type_ids, label

In [20]:
# Create dataloaders
train_dataset = CustomDataset(train_inputs, df_train_pd['label'].tolist())
# val_dataset = CustomDataset(val_inputs, df_val_pd['label'].tolist())
# test_dataset = CustomDataset(test_inputs, df_test_pd['label'].tolist())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [21]:
# Instantiate CNN model
cnn_model = CNNModel(input_size=128)

In [22]:
# Instantiate BERT-CNN ensemble model
ensemble_model = BERTCNNEnsemble(bert_model, cnn_model)

In [23]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(ensemble_model.parameters(), lr=learning_rate)

In [24]:
# Training loop
for epoch in range(num_epochs):
    ensemble_model.train()
    for batch in train_loader:
        input_ids, attention_mask, token_type_ids, labels = batch
        optimizer.zero_grad()
        outputs = ensemble_model(input_ids, attention_mask, token_type_ids)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

RuntimeError: Given groups=1, weight of size [128, 128, 3], expected input[1, 32, 768] to have 128 channels, but got 32 channels instead

In [ ]:
# Evaluation loop
ensemble_model.eval()
val_preds = []
val_targets = []
with torch.no_grad():
    for batch in val_loader:
        input_ids, attention_mask, token_type_ids, labels = batch
        logits = ensemble_model(input_ids, attention_mask, token_type_ids)
        _, predicted = torch.max(logits, 1)
        val_preds.extend(predicted.tolist())
        val_targets.extend(labels.tolist())

# Calculate performance scores
precision = precision_score(val_targets, val_preds)
recall = recall_score(val_targets, val_preds)
f1 = f1_score(val_targets, val_preds)
f_beta = fbeta_score(val_targets, val_preds, beta=0.5)

print("Validation Performance Scores:")
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
print("F1 Beta Score:", f_beta)

#### Build ML Pipeline with pretrained BERT model

In [13]:
# Step 1: Transforms raw texts to `document` annotation
document = DocumentAssembler()\
              .setInputCol("merged_text")\
              .setOutputCol("document")\
              .setCleanupMode("shrink")

# Step 2: Encodes text into high dimensional vectors
bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
              .setInputCols(["document"])\
              .setOutputCol("sentence_embeddings")

# Step 3: Encodes text into high-dimensional vectors using CNN - the current SentenceDetectorDLApproach uses CNN architecture
sentenceDetector = SentenceDetectorDLApproach() \
    .setInputCols(["document"]) \
    .setOutputCol("sentences") \
    .setEpochsNumber(100)

use_clf_pipeline = Pipeline(stages = [document,
                                      bert_sent,
                                      sentenceDetector])


TypeError: 'JavaPackage' object is not callable

In [ ]:
# Fit the training dataset to train the model
t0 = time.time()
pipelineModel = use_clf_pipeline.fit(trainingData)
print("Training time:", time.time()-t0)